In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import os
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sigmoid = torch.nn.Sigmoid()
torch.set_printoptions(precision=4)

In [89]:
MODEL_NAME = "unitary/unbiased-toxic-roberta"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model_config = AutoConfig.from_pretrained(MODEL_NAME)
labels = {k: v for k, v in sorted(model_config.label2id.items(), key=lambda item: item[1])}.keys()

In [93]:
input_data = {
 'inputs' : [
     "this is a very toxic comemnt. I want to offense everybody. Don't thread on me",
     "what is the differance between communist and pedophile",
     "Ok, this fucking Snowy is fucked me up",
     "I don't like SageMaker. It is not a great software tool. Only muslims could create such garbage",
     "Te quierro puta",
     "TOOL is the best rock band ever !!!111"
     ]   
}

In [94]:
sentences = input_data.pop("inputs", input_data)
inputs = tokenizer(sentences, padding=True, max_length=512, truncation=True, return_tensors='pt').to(device)
with torch.no_grad():
    model.to(device)
    outputs = model(**inputs).logits
    probas = sigmoid(outputs).cpu().detach().numpy().round(3).astype('str')
    res = [dict(zip(labels, probas[i])) for i in range(len(probas))]
    

In [95]:
json.dumps(res)

'[{"toxicity": "0.027", "severe_toxicity": "0.0", "obscene": "0.0", "identity_attack": "0.0", "insult": "0.016", "threat": "0.0", "sexual_explicit": "0.0", "male": "0.0", "female": "0.0", "homosexual_gay_or_lesbian": "0.0", "christian": "0.0", "jewish": "0.0", "muslim": "0.0", "black": "0.0", "white": "0.0", "psychiatric_or_mental_illness": "0.0"}, {"toxicity": "0.796", "severe_toxicity": "0.0", "obscene": "0.001", "identity_attack": "0.003", "insult": "0.679", "threat": "0.001", "sexual_explicit": "0.002", "male": "0.002", "female": "0.0", "homosexual_gay_or_lesbian": "0.0", "christian": "0.0", "jewish": "0.0", "muslim": "0.0", "black": "0.0", "white": "0.0", "psychiatric_or_mental_illness": "0.008"}, {"toxicity": "0.995", "severe_toxicity": "0.003", "obscene": "0.983", "identity_attack": "0.023", "insult": "0.947", "threat": "0.018", "sexual_explicit": "0.137", "male": "0.039", "female": "0.005", "homosexual_gay_or_lesbian": "0.002", "christian": "0.004", "jewish": "0.005", "muslim":